# Text Generation using LSTM Network
#### ディープラーニングによる文章生成

![title](https://camo.githubusercontent.com/9a5b885799c2d8e50f3f049fde2ada7696e974ca/68747470733a2f2f692e696d6775722e636f6d2f484646575674432e706e673f32)

## What is LSTM?
  
テキスト生成は一種の言語モデリング問題。  
言語モデリングは、テキスト読み上げ、会話システム、テキスト要約など、多数の自然言語処理タスクの中心的な問題。    
訓練された言語モデルは、テキストで使用されている前の一連の単語に基づいて、単語の出現の可能性を学習する。  
言語モデルは、文字レベル、n-gramレベル、文レベル、または段落レベルでも操作できる。  
このノートでは、最先端のリカレントニューラルネットワークを実装してトレーニングすることによって、自然言語テキストを生成するための言語モデルを作成する方法について説明していく
。

## 今回はニュースの本文からタイトルを自動生成します  
  
#### Process  
1. データの準備  
2. 文章のお掃除（記号削除、小文字統一）  
3. 単語に切り分ける  
4. トークン化＝数値化  
5. パディングで変数の長さを統一  
6. LSTMの実装

## 1. Import the libraries ライブラリのインポート

In [1]:
import pandas as pd
import numpy as np

## 2. Load the dataset データのロード

In [2]:
import os
path = os.getcwd()
print(path)

/Users/akr712/Desktop/今日やること/文章生成


In [3]:
ls

ArticlesApril2017.csv*               CommentsFeb2017.csv*
ArticlesApril2018.csv*               CommentsFeb2018.csv*
ArticlesFeb2017.csv*                 CommentsJan2017.csv*
ArticlesFeb2018.csv*                 CommentsJan2018.csv*
ArticlesJan2017.csv*                 CommentsMarch2017.csv*
ArticlesJan2018.csv*                 CommentsMarch2018.csv*
ArticlesMarch2017.csv*               CommentsMay2017.csv*
ArticlesMarch2018.csv*               maincode_text_generation_lstm.ipynb
ArticlesMay2017.csv*                 my_nltk.py
BoW.py                               mynltk.py
CommentsApril2017.csv*               nyt-comments/
CommentsApril2018.csv*               nytimes.ipynb


In [4]:
headlines = []
for filename in os.listdir(path):
    if "Articles" in filename:
        article_df = pd.read_csv(path + "/" + filename)
        headlines.extend(list(article_df["headline"].values))
        break
        
headlines = [ h for h in headlines if h != "Unknown" ]
print("The number of headline is :", len(headlines))

The number of headline is : 829


In [5]:
headlines

['N.F.L. vs. Politics Has Been Battle All Season Long',
 'Voice. Vice. Veracity.',
 'A Stand-Up’s Downward Slide',
 'New York Today: A Groundhog Has Her Day',
 'A Swimmer’s Communion With the Ocean',
 'Trail Activity',
 'Super Bowl',
 'Trump’s Mexican Shakedown',
 'Pence’s Presidential Pet',
 'Fruit of a Poison Tree',
 'The Peculiar Populism of Donald Trump',
 'Questions for: ‘On Alaska’s Coldest Days, a Village Draws Close for Warmth’',
 'The New Kids',
 'What My Chinese Mother Made',
 'Do You Think Teenagers Can Make a Difference in the World?',
 'President Pledges to Let Politics Return to Pulpits',
 'The Police Killed My Unarmed Son in 2012. I’m Still Waiting for Justice.',
 'Video of Sheep Slaughtering Ignites a Dispute',
 'This Will Change Your Mind',
 'Busy Start for a President, and That Was in 1933',
 'Trump Reverts to Pillars of Obama Policies Abroad',
 'Should Dollar Rise or Fall? The Trump Team’s Message Is Garbled',
 'Hardware for the Modern Winemaker',
 'Cajun, Far From H

## 3. Dataset preparation 前処理

### 3.1 Dataset cleaning  データクリーニング
  
記号を取り除き、「文字」と「数字」だけ残す。また小文字で統一する。

In [23]:
import string
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

def clean_text(headline):
    text = "".join( word for word in headline if word not in string.punctuation ).lower()
    text = text.encode("utf8").decode("ascii", "ignore")
    return text

# 元データのタイトルに含まれる単語群から独自のコーパスを作成
corpus = [ clean_text(headline) for headline in headlines ]

In [24]:
corpus

['nfl vs politics has been battle all season long',
 'voice vice veracity',
 'a standups downward slide',
 'new york today a groundhog has her day',
 'a swimmers communion with the ocean',
 'trail activity',
 'super bowl',
 'trumps mexican shakedown',
 'pences presidential pet',
 'fruit of a poison tree',
 'the peculiar populism of donald trump',
 'questions for on alaskas coldest days a village draws close for warmth',
 'the new kids',
 'what my chinese mother made',
 'do you think teenagers can make a difference in the world',
 'president pledges to let politics return to pulpits',
 'the police killed my unarmed son in 2012 im still waiting for justice',
 'video of sheep slaughtering ignites a dispute',
 'this will change your mind',
 'busy start for a president and that was in 1933',
 'trump reverts to pillars of obama policies abroad',
 'should dollar rise or fall the trump teams message is garbled',
 'hardware for the modern winemaker',
 'cajun far from home at the gumbo bros',
 '

### 3.2 Generating Sequence of N-gram Tokens 文章の単語化&数値化

- 自然言語処理では、テキストを単語単位に分解してベクトル化するのが主流である。  
- N-gram は、Morphological Analysis（形態要素解析）に並ぶ代表的な単語の切り出し手法のひとつ。  
- 具体的には、N-gramとは自然言語（テキスト）を連続するN個の文字、もしくはN個の単語単位で切り出す手法のこと。  
- 強みは「コーパス」が事前に入らないこと、弱みは切り出した単語数が肥大化しやすい点。  
- ex:) "I voted for Trump." n=2 => "I voted", "for Trump"

In [26]:
vocab = []
for line in corpus:
    words = line.split()
    for word in words:
        vocab.append(word)

vocabraly = set(vocab)

In [27]:
len(vocabraly)

2287

In [116]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(2000)
tokenizer.fit_on_texts(corpus)
word2index = tokenizer.word_index
len(word2index)

2287

In [117]:
word2index

{'rectum': 2031,
 'cancer': 322,
 'rick': 502,
 'tipoff': 2197,
 'sell': 1271,
 'shop': 1389,
 'call': 309,
 'preparing': 981,
 'areas': 794,
 'spacex': 1975,
 'houses': 2152,
 'to': 3,
 'upstart': 2074,
 'sheep': 356,
 'midnight': 1888,
 'president': 58,
 'scalpel': 823,
 'lounge': 851,
 'trapped': 1605,
 'wrist': 2171,
 'splits': 1210,
 'settlers': 1329,
 'learning': 1480,
 'streets': 400,
 'he': 116,
 'add': 1285,
 'speed': 268,
 'border': 2085,
 'card': 1524,
 'effect': 778,
 'consumer': 1101,
 'unarmed': 691,
 'vanish': 2202,
 'short': 886,
 'them': 574,
 'learned': 2011,
 'hate': 428,
 'anyway': 1120,
 'preening': 2068,
 'sugar': 435,
 'hacks': 1525,
 'croquemadame': 1877,
 'dusty': 1223,
 'betrayal': 1910,
 'counter': 1367,
 'go': 128,
 'mode': 1466,
 'pro': 1353,
 'highfive': 830,
 'finds': 1401,
 'muzzled': 1310,
 'much': 192,
 'isnt': 566,
 'violins': 1722,
 'treat': 2226,
 'uses': 640,
 'on': 9,
 'thats': 486,
 'readying': 800,
 'homeland': 307,
 'policy': 221,
 'civil': 118

In [118]:
dictionary = {}
rev_dictionary = {}
for word, idx in word2index.items():
    if idx > 1406:
        continue
    dictionary[word] = idx
    rev_dictionary[idx] = word

In [119]:
max(rev_dictionary.keys())

1406

In [120]:
input_seqences = tokenizer.texts_to_sequences(corpus)

In [121]:
input_seqences

[[660, 117, 72, 73, 661, 662, 63, 29, 210],
 [211, 663, 664],
 [2, 665, 666, 345],
 [11, 27, 28, 2, 667, 73, 153, 90],
 [2, 668, 669, 12, 1, 670],
 [346, 671],
 [212, 213],
 [19, 672, 673],
 [347, 348, 674],
 [675, 4, 2, 349, 676],
 [1, 677, 350, 4, 44, 8],
 [25, 6, 9, 678, 679, 351, 2, 680, 681, 214, 6, 682],
 [1, 11, 683],
 [14, 39, 352, 353, 684],
 [23, 13, 154, 215, 36, 91, 2, 685, 5, 1, 57],
 [58, 686, 3, 687, 72, 155, 3, 688],
 [1, 689, 690, 39, 691, 692, 5, 693, 216, 694, 354, 6, 118],
 [355, 4, 356, 695, 696, 2, 697],
 [30, 40, 119, 31, 120],
 [698, 74, 6, 2, 58, 7, 41, 75, 5, 357],
 [8, 699, 3, 700, 4, 92, 701, 217],
 [76, 702, 218, 32, 156, 1, 8, 703, 704, 10, 705],
 [706, 6, 1, 219, 707],
 [708, 121, 15, 45, 17, 1, 709, 358],
 [2, 220, 221, 710, 9, 359, 26, 157],
 [360, 711, 7, 712],
 [713, 6, 2, 93, 28, 7, 94, 361, 714],
 [1, 715, 362, 42],
 [42, 3, 716, 77, 158],
 [122, 717, 363, 7, 122, 159],
 [718, 7, 719, 720, 12, 222, 721],
 [722, 33, 364],
 [365, 95, 46, 37, 78, 9, 72

In [122]:
len(input_seqences)

829

### 3.3 Padding the Sequences and obtain Variables
#### パディングによって固定長データを作り、説明変数を得る

In [123]:
input_data = []
target = []
for line in input_seqences:
    for i in range(1, len(line)-1):
        input_data.append(line[:i])
        target.append(line[i+1])

In [124]:
input_data[:5]

[[660],
 [660, 117],
 [660, 117, 72],
 [660, 117, 72, 73],
 [660, 117, 72, 73, 661]]

In [125]:
target[:5]

[72, 73, 661, 662, 63]

In [126]:
MAX_LEN = 0
for seq in input_data:
    if len(seq) > MAX_LEN:
        MAX_LEN = len(seq)
MAX_LEN

15

In [127]:
input_data = pad_sequences(input_data, maxlen=MAX_LEN, padding="post", truncating="post")

In [128]:
len(input_data[0])

15

In [129]:
input_data.shape

(3455, 15)

In [130]:
target = to_categorical(target, num_classes=total_words)

In [131]:
target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [132]:
target.shape

(3455, 2288)

In [135]:
VOCAB_SIZE = 2001
VOCAB_SIZE

2001

In [151]:
MAX_LEN

15

## 4. LSTMs for Text Generation 長短期記憶層アルゴリズムの実装

### 4.1 LSTM ( Long Short-Term Memory  )    
  
1. Input Layer : Takes the sequence of words as input  
2. LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.  
3. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer.  
4. Output Layer : Computes the probability of the best possible next word as output  

![title](https://cdn-images-1.medium.com/max/1600/1*yBXV9o5q7L_CvY7quJt3WQ.png)

In [143]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping

In [138]:
model = Sequential()

"""入力層"""
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))

"""隠れ層"""
model.add(LSTM(units=100))
model.add(Dropout(rate=0.1))

"""出力層 活性化関数は多層のソフトマックス関数"""
model.add(Dense(units=target.shape[1], activation="softmax"))

In [139]:
"""損失関数と最適化手法の設定"""
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [140]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 15, 100)           200100    
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 2288)              231088    
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________


#### LSTMモデルにテストデータを学習させていく！

In [141]:
"""予測ラベルと正解ラベルを用意する"""
model.fit(input_data, target, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
3455/3455 [==============================] - 5s 1ms/step - loss: 7.2371
Epoch 2/5
3455/3455 [==============================] - 3s 940us/step - loss: 6.6598
Epoch 3/5
3455/3455 [==============================] - 3s 988us/step - loss: 6.5953
Epoch 4/5
3455/3455 [==============================] - 3s 932us/step - loss: 6.5657
Epoch 5/5
3455/3455 [==============================] - 3s 991us/step - loss: 6.5435


### 4.2 GRU ( Gated recurrent unit )

In [144]:
gru_model = Sequential()
gru_model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))
gru_model.add(GRU(units=100))
gru_model.add(Dropout(rate=0.1))
gru_model.add(Dense(units=target.shape[1], activation="softmax"))

In [145]:
gru_model.compile(loss="categorical_crossentropy", optimizer="adam")

In [146]:
gru_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 15, 100)           200100    
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2288)              231088    
Total params: 491,488
Trainable params: 491,488
Non-trainable params: 0
_________________________________________________________________


#### GRUモデルにテストデータを学習させていく！

In [147]:
gru_model.fit(input_data, target, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
3455/3455 [==============================] - 5s 1ms/step - loss: 7.2592
Epoch 2/5
3455/3455 [==============================] - 3s 882us/step - loss: 6.6681
Epoch 3/5
3455/3455 [==============================] - 3s 889us/step - loss: 6.5904
Epoch 4/5
3455/3455 [==============================] - 3s 885us/step - loss: 6.5545
Epoch 5/5
3455/3455 [==============================] - 3s 922us/step - loss: 6.5160


## 5. Generating the text タイトルにふさわしいテキストを自動生成する

In [142]:
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

In [152]:
def text_generater(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding="post")
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [157]:
text1 = "Trump decided"
text_generater(text1, 5, model, MAX_LEN)

'Trump Decided The The The The The'

In [158]:
text_generater(text1, 5, gru_model, MAX_LEN)

'Trump Decided The The The The The'

## I need more data to training I guess.... 学習不足